## ch13 Step-wise Tuning with Hyperopt
- https://github.com/mattharrison/effective_xgboost_book/blob/main/xgbcode.ipynb

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.8.3</div>
<div style="text-align: right"> Last update: 2023.8.3</div>

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

새롭게 라이브러리를 로드 하자.

In [2]:
import dtreeviz
from feature_engine import encoding, imputation
import numpy as np
import pandas as pd
from sklearn import base, compose, datasets, ensemble, \
    metrics, model_selection, pipeline, preprocessing, tree
import scikitplot
import xgboost as xgb
import yellowbrick.model_selection as ms
from yellowbrick import classifier

import urllib
import zipfile

import xg_helpers as xhelp
from xg_helpers import my_dot_export

In [3]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
'kaggle-survey-2018.zip'
fname = 'kaggle-survey-2018.zip'
member_name = 'multipleChoiceResponses.csv'

In [4]:
raw = xhelp.extract_zip(url, fname, member_name)
## Create raw X and raw y
kag_X, kag_y = xhelp.get_rawX_y(raw, 'Q6')

In [5]:
## Split data
kag_X_train, kag_X_test, kag_y_train, kag_y_test = \
model_selection.train_test_split(
kag_X, kag_y, test_size=.3, random_state=42, stratify=kag_y)

In [6]:
## Transform X with pipeline
X_train = xhelp.kag_pl.fit_transform(kag_X_train)
X_test = xhelp.kag_pl.transform(kag_X_test)
## Transform y with label encoder
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(kag_y_train)
y_train = label_encoder.transform(kag_y_train)
y_test = label_encoder.transform(kag_y_test)
# Combined Data for cross validation/etc
X = pd.concat([X_train, X_test], axis='index')
y = pd.Series([*y_train, *y_test], index=X.index)

In [7]:
X.head()

,age,education,years_exp,compensation,python,r,sql,Q1_Male,Q1_Female,Q1_Prefer not to say,Q1_Prefer to self-describe,Q3_United States of America,Q3_India,Q3_China,major_cs,major_other,major_eng,major_stat
587,25,18.0,4.0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0
3065,22,16.0,1.0,10000,1,0,0,1,0,0,0,0,1,0,1,0,0,0
8435,22,18.0,1.0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
3110,40,20.0,3.0,125000,1,0,1,0,1,0,0,1,0,0,0,1,0,0
16372,45,12.0,5.0,100000,1,0,1,1,0,0,0,1,0,0,0,1,0,0


In [8]:
y.head()

587      1
3065     0
8435     0
3110     0
16372    1
dtype: int64

여기서는 step-wise 튜닝 메서드를 소개한다.   
이 방법은 시간이 절약된다.   
이방법은 유사하게 동작하는 작은 하이퍼파라미터 그룹을 튜닝하고 그 값을 유지한 상태에서 다음 그룹으로 이동하는 방식이다.  
앞과 비교해서 서치 스페이스를 줄여서 시간을 단축시킨다.

### 13.1 Groups of Hyperparameters

여기서는 하이퍼파라미터를 작은 그룹으로 구분하고, 튜닝 후 다음으로 옮기는 방식을 사용한다.   
튜닝 그룹은 다음과 같다.   

- Tree parameters  
- Sampling parameters  
- Regularization parameters  
- Learning rate  

이 방법은 서치스페이스를 줄일 수 있다.  
물론 로컬 맥시멈에 빠질 수 있고 하이퍼파라미터간 상호작용을 놓칠수도 있다. 그러나 시간과의 트레이드오프를 고려했을 때 고려해볼만한다.  

다음은 step-wise 튜닝을 위한 코드이다.   
rounds 리스트에 각 스텝에 평가할 하이프파라미터 딕셔너리가 포함되어 있다.  
fmin의 max_evals는 round ehdgks hyperopt가 시도하는 횟수이다.  
더 많은 값을 조사하고 싶으면 이 숫자를 늘리면 된다.

In [9]:
from hyperopt import fmin, tpe, hp, Trials

In [10]:
params = {'random_state': 42}

rounds = [{'max_depth': hp.quniform('max_depth', 1, 8, 1),  # tree
           'min_child_weight': hp.loguniform('min_child_weight', -2, 3)},
          {'subsample': hp.uniform('subsample', 0.5, 1),   # stochastic
           'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)},
          {'reg_alpha': hp.uniform('reg_alpha', 0, 10),
            'reg_lambda': hp.uniform('reg_lambda', 1, 10),},
          {'gamma': hp.loguniform('gamma', -10, 10)}, # regularization
          {'learning_rate': hp.loguniform('learning_rate', -7, 0)} # boosting
]

In [11]:
all_trials = []
for round in rounds:
    params = {**params, **round}
    trials = Trials()
    best = fmin(fn=lambda space: xhelp.hyperparameter_tuning(space, X_train, 
                                        y_train, X_test, y_test),            
        space=params,           
        algo=tpe.suggest,            
        max_evals=20,            
        trials=trials,
    )
    params = {**params, **best}
    all_trials.append(trials)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 20/20 [00:02<00:00,  9.38trial/s, best loss: -0.7646408839779005]


- 학습하는데 시간이 많이 줄었다.

### 13.2 Visualization Hyperparameter Scores

In [12]:
all_trials

앞에서 작업한 것들 중에 reg_alpha, reg_lambda의 관계를 시각화해보자.

In [13]:
xhelp.plot_3d_mesh(xhelp.trial2df(all_trials[2]),
'reg_alpha', 'reg_lambda', 'loss')

- 이전 플롯보다 훨씬 더 거칠다. 
- 이를 사용하여 max_evals 수를 늘려야 하는지 진단할 수 있다.

### 13.3 Training an Optimized Model

최적화된 매개변수를 준비했으니 이제 이를 사용해 모델을 훈련해 봅시다. 값을 명시적으로 인쇄해야 합니다(이렇게 하면 노트북을 다시 시작해야 할 때 검색 공간을 다시 실행하지 않아도 되므로 도움이 됩니다).

In [14]:
params

{'random_state': 42,
 'max_depth': 3.0,
 'min_child_weight': 1.4108060075550906,
 'subsample': 0.5992813995218118,
 'colsample_bytree': 0.9296718702583378,
 'reg_alpha': 9.188145313451017,
 'reg_lambda': 4.390566392630925,
 'gamma': 0.0005338839113042646,
 'learning_rate': 0.44059094597958526}

In [15]:
# 주의 max_depth를 int로 설정해야함
step_params = {'random_state': 42,
 'max_depth': 6,
 'min_child_weight': 0.41163443122762267,
 'subsample': 0.8311146943629781,
 'colsample_bytree': 0.7316124626355548,
 'reg_alpha': 7.7808978956754205,
 'reg_lambda': 8.573634609663944,
 'gamma': 0.0013365573417158927,
 'learning_rate': 0.42895908657902554}

이제 학습을 진행해본다.

In [16]:
xg_step = xgb.XGBClassifier(**step_params, early_stopping_rounds=50,
                            n_estimators=500)
xg_step.fit(X_train, y_train,
       eval_set=[(X_train, y_train),
                 (X_test, y_test)
                ],
        verbose=100
      )

[0]	validation_0-logloss:0.62663	validation_1-logloss:0.62506


[100]	validation_0-logloss:0.47625	validation_1-logloss:0.49199
[200]	validation_0-logloss:0.46526	validation_1-logloss:0.49069
[227]	validation_0-logloss:0.46357	validation_1-logloss:0.49098


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7316124626355548, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0013365573417158927, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.42895908657902554, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=0.41163443122762267, missing=nan,
              monotone_constraints=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=42, ...)

In [17]:
xg_step.score(X_test, y_test)

0.7613259668508288

디펄트 모델과 비교해보자.

In [18]:
xg_def = xgb.XGBClassifier()
xg_def.fit(X_train, y_train)
xg_def.score(X_test, y_test)

0.7458563535911602